In [1]:
%matplotlib inline
import pandas as pd
from sqlalchemy import create_engine # database connection
import os
import datetime as dt
from IPython.display import display

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Bar, Scatter, Figure, Layout
import plotly.plotly as py
init_notebook_mode(connected=True)

In [2]:
def getDir(dir = ''):
    directory = r'C:/Data/Repos/Samspill/Dataset/'
    if len(dir) > 0:
        directory = directory + dir + '/'
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    return directory

In [3]:
result = getDir('powerbi')
writer = pd.ExcelWriter(result + r'samspill.xlsx', engine='xlsxwriter')
#i_n.to_excel(writer, sheet_name='')
#writer.save()

In [4]:
disk_engine = create_engine('sqlite:///C:/Data/Data/samspill.db') 

# Innovasjon Norge Vedtak

In [5]:
in_excel = pd.ExcelFile(getDir() + r'IN_17_03_17.xlsx',encoding='utf-8')
in_excel.sheet_names

['Data', 'DIM_vmkode']

In [6]:
in_data = in_excel.parse('Data')
in_data.drop('#Vm-kode', 1, inplace=True)
in_data.rename(columns={'Endelig innvilget årsrapport beløp':'Beløp'}, inplace=True)
in_data.head(1)

,Year,Organisasjonsnummer,Prosjekt kundenavn,Saknummer,Virkemiddelkode,Beløp
0,2012,00005001512,IN Møre og Romsdal,2011/102608,703 - Bedriftsutviklingstilskudd,67200.0


In [7]:
in_kode = in_excel.parse('DIM_vmkode')
in_kode.drop('#vmkode', 1, inplace=True)
in_kode.rename(columns={'Behov-NY':'Behov', 'Tjeneste-Ny': 'Tjeneste', 'Leveransetype-NY': 'Leveransetype' }, inplace=True)
in_kode.head(1)

,Virkemiddelkode,Tjenestetype,Tjenestekode,Virkemiddelgruppe,Behov,Tjeneste - NY,Leveransetype - NY,Budsjettpost,Myndighet
0,001 - Prosjekter/programmer,Andre tjenester,P-005 - Bygdeutviklingsmidlene,Sentrale BU-midler,Finansiere innovasjon og annen bedriftsutvikling,Investeringsstøtte,Tilskudd,LMD 1150.50 BU-midler,LMD


In [28]:
in_kode['Virkemiddelkode','Tjenestetype'].groupby(['Virkemiddelkode'])

KeyError: ('Virkemiddelkode', 'Tjenestetype')

In [8]:
in_data = pd.merge(in_data, in_kode, how='left', on='Virkemiddelkode')

In [9]:
in_data.head(1)

,Year,Organisasjonsnummer,Prosjekt kundenavn,Saknummer,Virkemiddelkode,Beløp,Tjenestetype,Tjenestekode,Virkemiddelgruppe,Behov,Tjeneste - NY,Leveransetype - NY,Budsjettpost,Myndighet
0,2012,00005001512,IN Møre og Romsdal,2011/102608,703 - Bedriftsutviklingstilskudd,67200.0,Finansieringstjenester,TF-054 - Tilskudd,Bedriftsutviklingstilskudd,Finansiere innovasjon og annen bedriftsutvikling,Investeringsstøtte,Tilskudd,KMD 551.60 Tilskudd til fylkeskommuner for reg...,KMD


In [10]:
in_data.to_sql('in', disk_engine, if_exists='replace')

# Eniro

In [11]:
datafil = r'C:\Data\Eniro\NFR_LEV_170203\SelskaperMedRegnskap_030217.txt'

In [12]:
pd.read_csv(datafil, nrows=1, sep='\t', encoding='utf-8')

,﻿orgnr,BEDRIFTSNR,Juridisk selskapsnavn,orgform,organisasjonsform,Status,Foretakets markedsnavn,Tidligere Foretaksnavn,fadresse,fpostnr,...,INGEN_DRIFT,UNDER_AVVIKLING,AVVIKENDE_REGNSKAPSAAR,EK_MINDRE_ENN_50_AV_AK,EGENKAPITALEN_ER_TAPT,AKSJEKAPITALEN_ER_TAPT,Revisor_orgnr,Revisor_Navn,Revisor_Postnr,Revisor_Poststed
0,810006862,NaN,WMF AS,AS,Aksjeselskap,Oppløst,WMF AS,WILHELMSEN MARINE FUELS AS,Strandveien 20,1366,...,0,0,0,0,0,0,933922847,PRICEWATERHOUSECOOPERS DA,278,OSLO


In [13]:
def toSqlLite(file, table):
    start = dt.datetime.now()
    chunksize = 20000
    j = 0
    index_start = 1

    #disk_engine.execute('DROP TABLE ' + table)

    # obs!! dropper linjer med feil (error_bad_lines=False)
    for df in pd.read_csv(file, sep='\t', 
                          encoding='utf-8',\
                          #names=oppsett.columns,\
                          chunksize=chunksize, 
                          iterator=True,\
                          error_bad_lines=False,
                          low_memory=False):

        df = df.rename(columns={c: c.replace(' ', '_') for c in df.columns}) # Remove spaces from columns
        df = df.rename(columns={c: c.replace('﻿', '') for c in df.columns}) # Remove spaces from columns

        df.index += index_start

        j+=1
        print('{} seconds: completed {} rows'.format((dt.datetime.now() - start).seconds, j*chunksize))

        df.to_sql(table, disk_engine, if_exists='append')
        index_start = df.index[-1] + 1

In [14]:
toSqlLite(r'C:\Data\Eniro\NFR_LEV_170203\SelskaperMedRegnskap_030217.txt', 'med')

0 seconds: completed 20000 rows
2 seconds: completed 40000 rows
5 seconds: completed 60000 rows
7 seconds: completed 80000 rows
9 seconds: completed 100000 rows
12 seconds: completed 120000 rows
14 seconds: completed 140000 rows
17 seconds: completed 160000 rows
19 seconds: completed 180000 rows
21 seconds: completed 200000 rows
24 seconds: completed 220000 rows
26 seconds: completed 240000 rows
28 seconds: completed 260000 rows
30 seconds: completed 280000 rows
33 seconds: completed 300000 rows
36 seconds: completed 320000 rows
38 seconds: completed 340000 rows
41 seconds: completed 360000 rows
44 seconds: completed 380000 rows
46 seconds: completed 400000 rows
49 seconds: completed 420000 rows
51 seconds: completed 440000 rows
53 seconds: completed 460000 rows
56 seconds: completed 480000 rows
58 seconds: completed 500000 rows
60 seconds: completed 520000 rows
63 seconds: completed 540000 rows
65 seconds: completed 560000 rows
68 seconds: completed 580000 rows
70 seconds: completed 6

In [15]:
toSqlLite(r'C:\Data\Eniro\NFR_LEV_170203\SelskaperUtenRegnskap_030217.txt', 'uten')

0 seconds: completed 20000 rows
2 seconds: completed 40000 rows
5 seconds: completed 60000 rows
7 seconds: completed 80000 rows
9 seconds: completed 100000 rows
12 seconds: completed 120000 rows
14 seconds: completed 140000 rows
17 seconds: completed 160000 rows
19 seconds: completed 180000 rows
21 seconds: completed 200000 rows
23 seconds: completed 220000 rows
26 seconds: completed 240000 rows
28 seconds: completed 260000 rows
30 seconds: completed 280000 rows
33 seconds: completed 300000 rows
35 seconds: completed 320000 rows
37 seconds: completed 340000 rows
40 seconds: completed 360000 rows
42 seconds: completed 380000 rows
44 seconds: completed 400000 rows
47 seconds: completed 420000 rows
49 seconds: completed 440000 rows
52 seconds: completed 460000 rows
54 seconds: completed 480000 rows
56 seconds: completed 500000 rows
59 seconds: completed 520000 rows
61 seconds: completed 540000 rows
63 seconds: completed 560000 rows
66 seconds: completed 580000 rows
68 seconds: completed 6

In [16]:
pd.read_sql_query('SELECT * FROM med LIMIT 1', disk_engine)

,index,orgnr,BEDRIFTSNR,Juridisk_selskapsnavn,orgform,organisasjonsform,Status,Foretakets_markedsnavn,Tidligere_Foretaksnavn,fadresse,...,INGEN_DRIFT,UNDER_AVVIKLING,AVVIKENDE_REGNSKAPSAAR,EK_MINDRE_ENN_50_AV_AK,EGENKAPITALEN_ER_TAPT,AKSJEKAPITALEN_ER_TAPT,Revisor_orgnr,Revisor_Navn,Revisor_Postnr,Revisor_Poststed
0,1,810006862,None,WMF AS,AS,Aksjeselskap,Oppløst,WMF AS,WILHELMSEN MARINE FUELS AS,Strandveien 20,...,0.0,0.0,0.0,0.0,0.0,0.0,933922847.0,PRICEWATERHOUSECOOPERS DA,278.0,OSLO


In [17]:
pd.read_sql_query('SELECT * FROM uten LIMIT 1', disk_engine)

,index,orgnr,BEDRIFTSNR,Juridisk_selskapsnavn,orgform,organisasjonsform,Status,Foretakets_markedsnavn,Tidligere_Foretaksnavn,fadresse,...,INGEN_DRIFT,UNDER_AVVIKLING,AVVIKENDE_REGNSKAPSAAR,EK_MINDRE_ENN_50_AV_AK,EGENKAPITALEN_ER_TAPT,AKSJEKAPITALEN_ER_TAPT,Revisor_orgnr,Revisor_Navn,Revisor_Postnr,Revisor_Poststed
0,1,810061642,None,JOHAN AASEN innehaver Kristin Aasen,ENK,Enkeltpersonforetak,Slettet,AASEN JOHAN INNEHAVER KRISTIN AASEN,None,Omveien 18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
#sql = 'DROP TABLE eniro'
#disk_engine.execute(sql)

sql = 'CREATE TABLE eniro AS SELECT * FROM med WHERE 0'
disk_engine.execute(sql)

In [20]:
sql = 'INSERT INTO eniro SELECT * FROM (SELECT * FROM med UNION ALL SELECT * FROM uten)'
disk_engine.execute(sql)

In [21]:
eniro = pd.read_sql_query('SELECT * FROM eniro LIMIT 5', disk_engine)

In [22]:
eniro.dtypes

index                        int64
orgnr                        int64
BEDRIFTSNR                  object
Juridisk_selskapsnavn       object
orgform                     object
organisasjonsform           object
Status                      object
Foretakets_markedsnavn      object
Tidligere_Foretaksnavn      object
fadresse                    object
fpostnr                    float64
fpoststed                   object
Kommune                     object
Fylke                       object
padresse                    object
ppostnr                    float64
ppoststed                   object
konsernspiss_orgnr          object
konsernspiss_navn           object
datterselskaper             object
ORGNRPARENT                 object
Underavdelinger             object
Aksjeeiere                  object
URL                         object
EPOST                       object
BRANSJEKODE                float64
BRANSJE                     object
NACE2_SN07                  object
NACE3_SN07          

In [23]:
#sql = 'DROP TABLE eniro_siste'
#disk_engine.execute(sql)


In [24]:
sql = """
        INSERT INTO eniro_siste
        SELECT * FROM 
        (
            SELECT e.* 
            FROM eniro AS e JOIN 
            (
                SELECT orgnr, MAX(Regnaar) AS Regnaar from eniro GROUP BY orgnr
            ) AS f 
            ON e.Regnaar = f.Regnaar AND e.orgnr = f.orgnr
        )
    """

disk_engine.execute(sql)

OperationalError: (sqlite3.OperationalError) no such table: eniro_siste [SQL: '\n        INSERT INTO eniro_siste\n        SELECT * FROM \n        (\n            SELECT e.* \n            FROM eniro AS e JOIN \n            (\n                SELECT orgnr, MAX(Regnaar) AS Regnaar from eniro GROUP BY orgnr\n            ) AS f \n            ON e.Regnaar = f.Regnaar AND e.orgnr = f.orgnr\n        )\n    ']

In [25]:
sql = """
    SELECT * FROM [in] AS i LEFT JOIN 
    (
    SELECT e.orgnr,
           e.Juridisk_selskapsnavn,
           e.STIFTDATO,
           e.REGDATO,
           e.SLETDATO,
           e.Fylke,
           e.Bransje,
           e.Antall_ansatte,
           e.Antall_ansatte_konsern,
           e.ANTALL_AARSVERK,
           e.ANTALL_AARSVERK_KONSERN,
           e.Regnaar, 
           e.LATITUDE, 
           e.LONGITUDE, 
           e.SEKTOR_2014 AS 'Sektor' 
    FROM eniro_siste AS e 
    ON i.Organisasjonsnummer = e.orgnr
    """


In [26]:
in_eniro = pd.read_sql_query(sql, disk_engine)

OperationalError: (sqlite3.OperationalError) near "
    ": syntax error [SQL: "\n    SELECT * FROM [in] AS i LEFT JOIN \n    (\n    SELECT e.orgnr,\n           e.Juridisk_selskapsnavn,\n           e.STIFTDATO,\n           e.REGDATO,\n           e.SLETDATO,\n           e.Fylke,\n           e.Bransje,\n           e.Antall_ansatte,\n           e.Antall_ansatte_konsern,\n           e.ANTALL_AARSVERK,\n           e.ANTALL_AARSVERK_KONSERN,\n           e.Regnaar, \n           e.LATITUDE, \n           e.LONGITUDE, \n           e.SEKTOR_2014 AS 'Sektor' \n    FROM eniro_siste AS e \n    ON i.Organisasjonsnummer = e.orgnr\n    "]

In [27]:
in_eniro[in_eniro['Antall_ansatte'].notnull()].head(100)

NameError: name 'in_eniro' is not defined

In [ ]:
sql = """
    SELECT * FROM [in] WHERE Virkemiddelkode= '922 - Marint innovasjonsprogram'
    """

pd.read_sql_query(sql, disk_engine)

In [ ]:
in_data[(in_data['Virkemiddelkode']=='922 - Marint innovasjonsprogram')]

In [ ]:
in_kode = in_excel.parse('DIM_vmkode')
in_kode[(in_kode['Virkemiddelkode']=='922 - Marint innovasjonsprogram')]